In [1]:
from librium.database import *
from csv import DictReader
import re
from sqlalchemy.orm.exc import NoResultFound

In [5]:
with open("books.csv") as fp:
    books = [x for x in DictReader(fp)]

In [8]:
for book in books:
    # db_book = Book.query.filter(Book.isbn == book["isbn"], Book.title == book["title"]).one()
    hardcover = Format.query.filter(Format.name == "Hard cover").first()
    paperback = Format.query.filter(Format.name == "Paperback").first()

    db_book = Book(
        title=book["title"],
        isbn=book["isbn"],
        page_count=book["pages"],
        price=float(book["list_price"]),
        read=book["read"] == 1,
        format=hardcover if book["format"] == "Pevná vazba" else paperback
    )

    languages = [x.strip() for x in book["language"].split(",")]
    for language in languages:
        db_book.add(db_book.languages, Language, {"name": language})
    publishers = [x.strip() for x in book["publisher"].split(",")]
    for publisher in publishers:
        db_book.add(db_book.publishers, Publisher, {"name": publisher})

    genre = book["genre"]
    db_book.add(db_book.genres, Genre, {"name": genre})

    authors_raw = book["author_details"].split("|")
    for author in authors_raw:
        _ = author.split(", ")
        first_name, last_name = _[::-1]
        f_author = " ".join((first_name, last_name)).strip()
        db_author = Author.query.filter(Author.name == f_author).one_or_none()
        if not db_author:
            db_author = Author(
                first_name=first_name,
                last_name=last_name
            )
        db_book.authors.append(db_author)

    db_session.commit()

    if book["series_details"]:
        series_pattern = re.compile(r"(?P<title>[^(]+)( ?\((?P<idx>[0-9.]+)\))?")
        series_raw = book["series_details"].split("|")
        for series in series_raw:
            srch = series_pattern.fullmatch(series)
            title = srch.group("title").strip()
            idx = float(srch.group("idx")) if srch.group("idx") else 0
            try:
                db_series = Series.query.filter(Series.name == title).one()
            except NoResultFound:
                db_series = Series(name=title)
                db_session.commit()
            try:
                db_series_index = SeriesIndex.query.filter(SeriesIndex.series == db_series).filter(SeriesIndex.book == db_book).one()
                if db_series_index.idx != idx:
                    db_series_index.idx = idx
            except NoResultFound:
                db_series_index = SeriesIndex(series=db_series, book=db_book, idx=idx)
                db_book.series.append(db_series_index)
            print(db_series_index)

    db_session.commit()

SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=1.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=17.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=20.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=13.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=23.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=5.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=4.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=1.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=2.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=6.0)
SeriesIndex(book_id=<not loaded>, series_id=<not loaded>, idx=7.0)
